### 외부 데이터 공유 드라이브 :
https://drive.google.com/drive/folders/1T3mcWrV4rK8dDyBQk7LnQS6k_YFSCB2r?usp=sharing 

위의 공유 드라이브에서 데이터 다운로드가 안될 경우 아래의 다운로드 방법 사용 

외부 데이터 다운로드 방법 :
1. 아래 링크에 들어감
https://kr.investing.com/indices/south-korea-indices?&primarySectors=on&otherIndices=on

    - 아래 종류 선택    
    KOSDAQ Machinery, KRX Transportation, KRX Steel, KRX Media & Telecom, KRX Health Care, KRX Energy & Chemical, KRX Constructions, KRX Semiconductor, KRX Bank, KOSDAQ Entertainment & Culture, KOSDAQ Manufacturing, KOSPI 200 IT

    - 윗쪽 부분에 있는 '과거 데이터' 선택
    - '데이터 다운로드' 버튼 옆의 날짜 선택 버튼 클릭
    - 날짜 지정 시작일자 : 2016/01/01, 종료 일자 : 2020/12/31
    - '신청합니다' 버튼 클릭
    -  '데이터 다운로드' 버튼 클릭
    - 위의 과정을 2번 항목의 데이터들에 대해 반복 수행


2. 아래 링크에 들어감
http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201
    - 왼쪽 메뉴바에서 '지수' -> '주가지수' -> '개별지수 시세추이' 선택
    - 지수명에서 종류 검색 : 코스피200, 코스닥150, 코스피200제외 코스피지수
    - 아래 조회기간 날짜 지정 : 2016/01/04 ~ 2020/12/30
    - '조회' 클릭
    - '조화' 버튼 아래 파란색 '다운로드' 클릭 후 csv 파일 다운로드<br><br>
    - <span style="color:red">직접 다운로드 시,  csv 파일의 경우 다운받은 날짜에 따라 이름이 바뀌어 파일 경로의 파일명도 변경해줘야 함</span>

In [1]:
import sys
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os
import sklearn
import re
import matplotlib
import tqdm as tq
import copy
import lightgbm

import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import KFold
from lightgbm import LGBMRegressor
from sklearn.metrics import explained_variance_score
from tqdm import tqdm

In [2]:
! pip install pandas
! pip install matplotlib
! pip install tqdm
! pip install sklearn
! pip install numpy

### 개발 OS 환경

- 개발 환경(OS): Windows 10 Pro, 64비트 운영 체제, x64 기반 프로세서

### Library version 확인

-------------------------- Python & library version --------------------------
- Python version: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
- pandas version: 1.2.0
- numpy version: 1.19.2
- sklearn version: 0.23.2
- matplotlib version: 3.3.2
- tqdm version: 4.62.2
- lightgbm version: 3.1.1
------------------------------------------------------------------------------

In [3]:
print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("sklearn version: {}".format(sklearn.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("lightgbm version: {}".format(lightgbm.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.2.4
numpy version: 1.20.1
sklearn version: 0.24.1
matplotlib version: 3.3.4
tqdm version: 4.59.0
lightgbm version: 3.2.1
------------------------------------------------------------------------------


### 주어진 데이터 및 외부 데이터

In [4]:
cus = pd.read_csv("data/cus_info.csv")
iem = pd.read_csv("data/iem_info_20210902.csv")
hist = pd.read_csv("data/stk_bnc_hist.csv")
train = pd.read_csv("data/stk_hld_train.csv")
test = pd.read_csv("data/stk_hld_test.csv")
submission = pd.read_csv("data/sample_submission.csv")

# 코스피 200 지수
kospi = pd.read_csv('data/data_5320_20211011.csv', encoding='cp949')
# 코스닥 150 지수
kosdaq = pd.read_csv('data/data_5419_20211011.csv', encoding='cp949')
# 코스피 200제외 코스피 지수
other = pd.read_csv('data/data_5517_20211011.csv', encoding='cp949')

# 건설 : 01
construction_krx =  pd.read_csv("data/KRX Constructions 내역.csv", thousands = ',')
# 금융 : 02
bank_krx =  pd.read_csv("data/KRX Bank 내역.csv", thousands = ',')
# 기계 : 03
machine_krx = pd.read_csv("data/KOSDAQ Machinery 내역.csv", thousands = ',' )
#통신 : 04 
telecom_krx =  pd.read_csv("data/KRX Media & Telecom 내역.csv", thousands = ',')
# 서비스 : 05
entertainment_kosdaq =   pd.read_csv("data/KOSDAQ Entertainment & Culture 내역.csv", thousands = ',')
# 운송 : 06
transportation_krx= pd.read_csv("data/KRX Transportation 내역.csv", thousands = ',')
# 유통 : 07
transportation_krx= pd.read_csv("data/KRX Transportation 내역.csv", thousands = ',')
# 의료 : 08
medical_krx =  pd.read_csv("data/KRX Health Care 내역.csv", thousands = ',')
# 전기 : 09
electric_krx =  pd.read_csv("data/KRX Semiconductor 내역.csv", thousands = ',')
# 제조 : 10
manufacturing_kosdaq =   pd.read_csv("data/KOSDAQ Manufacturing 내역.csv", thousands = ',')
# 철강 : 11
steel_krx= pd.read_csv("data/KRX Steel 내역.csv", thousands = ',')
# 화학 : 12
chemical_krx =  pd.read_csv("data/KRX Energy & Chemical 내역.csv", thousands = ',')
# IT : 13
IT_kospi =   pd.read_csv("data/KOSPI 200 IT 내역.csv", thousands = ',')

# 기타 : 14
# 종가만 사용하기 때문에 임이의 KRX 건설(01) csv파일을 가져온 후 모든 업종 지수의 종가 평균을 구함
etc_krx = pd.read_csv("data/KRX Constructions 내역.csv", thousands = ',')
etc_krx['종가'] = etc_krx['종가'] + bank_krx['종가'] + machine_krx['종가'] + telecom_krx['종가'] + entertainment_kosdaq['종가'] + transportation_krx['종가'] + transportation_krx['종가'] +medical_krx['종가'] + electric_krx['종가'] + manufacturing_kosdaq['종가'] + steel_krx['종가'] + chemical_krx['종가'] + IT_kospi['종가']
etc_krx['종가'] = etc_krx['종가'] /13

### 데이터 전처리

- 외부 데이터의 전처리

In [5]:
def data_cleansing(df):
    df["거래량"] = df["거래량"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
    df['날짜']=df['날짜'].str.replace(pat=r'[ㄱ-ㅣ가-힣]+', repl= r'', regex=True)
    df['날짜'] =df['날짜'].str.replace(" ","").astype(int)
    df['변동 %'] =df['변동 %'].str.replace("%","")
    df['거래량'] =df['거래량'].str.replace("B","0000000")
    df['거래량'] =df['거래량'].str.replace("M","0000")
    df['거래량'] =df['거래량'].str.replace("K","0")
    df['거래량'] = df['거래량'].astype(int)
    
def kos_data_cleansing(df):
    df['일자'] =df['일자'].str.replace("/","").astype(str)
    df['일자'] = df['일자'].astype(int)

In [6]:
data_cleansing(machine_krx)
data_cleansing(transportation_krx)
data_cleansing(steel_krx)
data_cleansing(telecom_krx)
data_cleansing(medical_krx)
data_cleansing(chemical_krx)
data_cleansing(construction_krx)
data_cleansing(electric_krx)
data_cleansing(bank_krx)
data_cleansing(entertainment_kosdaq)
data_cleansing(manufacturing_kosdaq)
data_cleansing(IT_kospi)
data_cleansing(etc_krx)

In [7]:
train["hist_d"] = train["hold_d"]*0.6
train.hist_d = np.trunc(train["hist_d"])

train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])

train_data['kos_idx'] = 0
test_data['kos_idx'] = 0

- 코스피200, 코스닥150, 기타(코스피200제외 코스피)의 사용하지 않는 컬럼 제외
- 3개의 데이터 프레임에서 같은 이름을 가진 컬럼의 이름을 구분하기 위해 이름 변경
- 3개의 데이터 프레임을 일자를 기준으로 하나의 데이터 프레임으로 합
- 데이터 전처리에서 자주 사용하는 컬럼 데이터 리스트에 저장 후 사용

In [8]:
kos_data_cleansing(kospi)
kos_data_cleansing(kosdaq)
kos_data_cleansing(other)

kospi = kospi.drop(['대비', '시가','고가','저가','거래량','거래대금','상장시가총액'], axis = 1)
kosdaq = kosdaq.drop(['대비', '시가','고가','저가','거래량','거래대금','상장시가총액'], axis = 1)
other = other.drop(['대비', '시가','고가','저가','거래량','거래대금','상장시가총액'], axis = 1)


kospi.rename(columns={'종가':'kospi'}, inplace=True)
kospi.rename(columns={'등락률':'kospi_idx'}, inplace=True)

kosdaq.rename(columns={'종가':'kosdaq'}, inplace=True)
kosdaq.rename(columns={'등락률':'kosdaq_idx'}, inplace=True)

other.rename(columns={'종가':'other'}, inplace=True)
other.rename(columns={'등락률':'other_idx'}, inplace=True)

train_data['kos_idx'] = 0
test_data['kos_idx'] = 0

byn_dt_train = list(train_data['byn_dt'])
hold_d_train = list(train_data['hold_d'])
stk_dit_cd_train = list(train_data['stk_dit_cd'])
byn_dt_test = list(test_data['byn_dt'])
hist_d_test = list(test_data['hist_d'])
stk_dit_cd_test = list(test_data['stk_dit_cd'])

kos_idx_train = []
kos_idx_test = []

kos_data = pd.merge(kospi, kosdaq, how= "left", on = ["일자"])
kos_data = pd.merge(kos_data, other, how= "left", on = ["일자"])
kos_data.rename(columns={'일자':'byn_dt'}, inplace=True)

#### 시장구분(stk_dit_cd) 기준 feature

- 새로운 feature을 만들기 위해 stk_dit_cd의 정보를 이용함
- 고객이 거래한 종목이 속한 시장과  매수일과 매도일을 바탕으로 해당 종목이 속한 시장의 지수 등락값을 구할 수 있음
- train_data와 test_data별로 인덱스에 맞게 새로 만들어준 kos_idx_train, kos_idx_test에 값을 저장하여 사용

- train_data에서 모든 데이터에 대한 지수 등락값의 분포를 그래프로 확인해본 결과 중앙값을 기준으로 정규분포로 분포함을 확인하였음
- 하지만 test_data 에서는 정규분포를 따르지 않았는데 이는 매도일을 정하는 과정에서 실제 보유일(hold_d)를 사용하는 것이 아닌 hist_d를 이용하여 그런것이라 생각함
- 따라서 test_data도 hold_d를 이용하여 얻은 지수 등락값의 분포를 확인하면 정규분포일 것이라는 가정
- test_data에서 각 데이터의 지수 등락값과 모든 데이터의 지수 등락값의 중앙값의 차이를 하루평균 지수 상승량으로 나누어 주어 21년 보유일 (hold_d - hist_d)을 구하였음
- 이 과정에서 21년 보유일은 0~146일 이기 때문에 146 초과 및 0 미만의 경우 각각 146과 0으로 값을 보정

In [9]:
for i in tqdm(range(len(byn_dt_test)), desc = 'rate', mininterval = 0.01):
    if stk_dit_cd_test[i] == 1:
        kos_idx_test.append((kos_data['kospi'][list(kos_data['byn_dt']).index(byn_dt_test[i])-hist_d_test[i]+1]-kos_data['kospi'][list(kos_data['byn_dt']).index(byn_dt_test[i])]))
        
    elif stk_dit_cd_test[i] ==2:
      
        kos_idx_test.append((kos_data['kosdaq'][list(kos_data['byn_dt']).index(byn_dt_test[i])-hist_d_test[i]+1]-kos_data['kosdaq'][list(kos_data['byn_dt']).index(byn_dt_test[i])]))
    else:
        kos_idx_test.append((kos_data['other'][list(kos_data['byn_dt']).index(byn_dt_test[i])-hist_d_test[i]+1]-kos_data['other'][list(kos_data['byn_dt']).index(byn_dt_test[i])]))

for i in tqdm(range(len(byn_dt_train)), desc = 'rate', mininterval = 0.01):
    if stk_dit_cd_train[i] == 1:
        kos_idx_train.append((kos_data['kospi'][list(kos_data['byn_dt']).index(byn_dt_train[i])-hold_d_train[i]+1]-kos_data['kospi'][list(kos_data['byn_dt']).index(byn_dt_train[i])]))
        
    elif stk_dit_cd_train[i] ==2:
      
        kos_idx_train.append((kos_data['kosdaq'][list(kos_data['byn_dt']).index(byn_dt_train[i])-hold_d_train[i]+1]-kos_data['kosdaq'][list(kos_data['byn_dt']).index(byn_dt_train[i])]))
    else:
        kos_idx_train.append((kos_data['other'][list(kos_data['byn_dt']).index(byn_dt_train[i])-hold_d_train[i]+1]-kos_data['other'][list(kos_data['byn_dt']).index(byn_dt_train[i])]))

rate: 100%|██████████████████████████████████████████████████████████████████| 681472/681472 [05:09<00:00, 2200.04it/s]


In [10]:
train_data['kos_idx'] = kos_idx_train
test_data['kos_idx'] = kos_idx_test

kos_per_day = (kos_data['kospi'][0] - kos_data['kospi'][1226])/len(kos_data)

test_kos_idx = copy.deepcopy(test_data['kos_idx'])
train_kos_idx = copy.deepcopy(train_data['kos_idx'])

test_kos_idx = (test_data['kos_idx'].max() + test_data['kos_idx'].min())/2 - test_kos_idx 
train_kos_idx = (train_data['kos_idx'].max() + train_data['kos_idx'].min())/2 - train_kos_idx 

test_kos_idx = test_kos_idx / kos_per_day
train_kos_idx = train_kos_idx / kos_per_day

for i in tqdm(range(len(test_kos_idx)), desc = 'rate', mininterval = 0.01):
    if test_kos_idx[i] < 0:
        test_kos_idx[i] = 0
        
for i in tqdm(range(len(train_kos_idx)), desc = 'rate', mininterval = 0.01):
    if train_kos_idx[i] < 0:
        train_kos_idx[i] = 0
        
test_kos_idx = test_kos_idx * (146/test_kos_idx.max())
train_kos_idx = train_kos_idx * (146/train_kos_idx.max())

test_kos_idx = np.trunc(test_kos_idx)
train_kos_idx = np.trunc(train_kos_idx)

train_data['kos_idx'] = train_kos_idx
test_data['kos_idx'] = test_kos_idx



rate: 100%|█████████████████████████████████████████████████████████████████| 681472/681472 [00:08<00:00, 82674.33it/s]


In [11]:
train_data

,act_id,iem_cd,byn_dt,hold_d,hist_d,sex_dit_cd,cus_age_stn_cd,ivs_icn_cd,cus_aet_stn_cd,mrz_pdt_tp_sgm_cd,lsg_sgm_cd,tco_cus_grd_cd,tot_ivs_te_sgm_cd,mrz_btp_dit_cd,iem_krl_nm,btp_cfc_cd,mkt_pr_tal_scl_tp_cd,stk_dit_cd,kos_idx
0,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A006360,20180726,11,6.0,1,9,3,2,2,9,5,5,8,GS건설,1,1,1,4.0
1,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005930,20180131,80,48.0,1,9,3,2,2,9,5,5,8,삼성전자,9,1,1,6.0
2,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A005070,20180517,5,3.0,1,9,3,2,2,9,5,5,8,코스모신소재,12,2,99,3.0
3,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A003520,20201112,22,13.0,1,9,3,2,2,9,5,5,8,영진약품,8,2,1,0.0
4,0ad104dbed99be0cd858aa772765ddedade554601a981b...,A002310,20180905,324,194.0,1,9,3,2,2,9,5,5,8,아세아제지,10,3,99,42.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
681467,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A260660,20180831,1,0.0,1,4,4,2,2,3,4,3,8,알리코제약,10,3,99,4.0
681468,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A271980,20201027,1,0.0,1,4,4,2,2,3,4,3,8,제일약품,8,2,99,4.0
681469,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A289080,20181121,1,0.0,1,4,4,2,2,3,4,3,8,SV인베스트먼트,2,2,99,4.0
681470,4561928f2825f6389f228088ac807a9fb4575ffdcbc47a...,A307930,20200214,1,0.0,1,4,4,2,2,3,4,3,8,컴퍼니케이,2,3,99,4.0


#### 종목 업종(btp_cfc_cd) 기준 feature
- 위에서는 종목이 속한 시장으로 기준을 나눴다면 아래에서는 종목의 업종을 기준으로 나눔
- 과정은 위와 동일

In [12]:
krx_idx_train = []
krx_idx_test = []

btp_cfc_cd_train = list(train_data['btp_cfc_cd'])
btp_cfc_cd_test = list(test_data['btp_cfc_cd'])

for i in tqdm(range(len(btp_cfc_cd_train)), desc = 'rate', mininterval = 0.01):
    if btp_cfc_cd_train[i] == 1:
        krx_idx_train.append((construction_krx['종가'][list(construction_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-construction_krx['종가'][list(construction_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 2:
        krx_idx_train.append((bank_krx['종가'][list(bank_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-bank_krx['종가'][list(bank_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 3:
        krx_idx_train.append((machine_krx['종가'][list(machine_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-machine_krx['종가'][list(machine_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 4:
        krx_idx_train.append((telecom_krx['종가'][list(telecom_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-telecom_krx['종가'][list(telecom_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 5:
        krx_idx_train.append((entertainment_kosdaq['종가'][list(entertainment_kosdaq['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-entertainment_kosdaq['종가'][list(entertainment_kosdaq['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 6:
        krx_idx_train.append((transportation_krx['종가'][list(transportation_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-transportation_krx['종가'][list(transportation_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 7:
        krx_idx_train.append((transportation_krx['종가'][list(transportation_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-transportation_krx['종가'][list(transportation_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 8:
        krx_idx_train.append((medical_krx['종가'][list(medical_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-medical_krx['종가'][list(medical_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 9:
        krx_idx_train.append((electric_krx['종가'][list(electric_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-electric_krx['종가'][list(electric_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 10:
        krx_idx_train.append((manufacturing_kosdaq['종가'][list(manufacturing_kosdaq['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-manufacturing_kosdaq['종가'][list(manufacturing_kosdaq['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 11:
        krx_idx_train.append((steel_krx['종가'][list(steel_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-steel_krx['종가'][list(steel_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 12:
        krx_idx_train.append((chemical_krx['종가'][list(chemical_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-chemical_krx['종가'][list(chemical_krx['날짜']).index(byn_dt_train[i])]))
    elif btp_cfc_cd_train[i] == 13:
        krx_idx_train.append((IT_kospi['종가'][list(IT_kospi['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-IT_kospi['종가'][list(IT_kospi['날짜']).index(byn_dt_train[i])]))        
    else:
        krx_idx_train.append((etc_krx['종가'][list(etc_krx['날짜']).index(byn_dt_train[i])-hold_d_train[i]+1]-etc_krx['종가'][list(etc_krx['날짜']).index(byn_dt_train[i])]))
        
for i in tqdm(range(len(btp_cfc_cd_test)), desc = 'rate', mininterval = 0.01):
    if btp_cfc_cd_test[i] == 1:
        krx_idx_test.append((construction_krx['종가'][list(construction_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-construction_krx['종가'][list(construction_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 2:
        krx_idx_test.append((bank_krx['종가'][list(bank_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-bank_krx['종가'][list(bank_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 3:
        krx_idx_test.append((machine_krx['종가'][list(machine_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-machine_krx['종가'][list(machine_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 4:
        krx_idx_test.append((telecom_krx['종가'][list(telecom_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-telecom_krx['종가'][list(telecom_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 5:
        krx_idx_test.append((entertainment_kosdaq['종가'][list(entertainment_kosdaq['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-entertainment_kosdaq['종가'][list(entertainment_kosdaq['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 6:
        krx_idx_test.append((transportation_krx['종가'][list(transportation_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-transportation_krx['종가'][list(transportation_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 7:
        krx_idx_test.append((transportation_krx['종가'][list(transportation_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-transportation_krx['종가'][list(transportation_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 8:
        krx_idx_test.append((medical_krx['종가'][list(medical_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-medical_krx['종가'][list(medical_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 9:
        krx_idx_test.append((electric_krx['종가'][list(electric_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-electric_krx['종가'][list(electric_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 10:
        krx_idx_test.append((manufacturing_kosdaq['종가'][list(manufacturing_kosdaq['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-manufacturing_kosdaq['종가'][list(manufacturing_kosdaq['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 11:
        krx_idx_test.append((steel_krx['종가'][list(steel_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-steel_krx['종가'][list(steel_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 12:
        krx_idx_test.append((chemical_krx['종가'][list(chemical_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-chemical_krx['종가'][list(chemical_krx['날짜']).index(byn_dt_test[i])]))
    elif btp_cfc_cd_test[i] == 13:
        krx_idx_test.append((IT_kospi['종가'][list(IT_kospi['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-IT_kospi['종가'][list(IT_kospi['날짜']).index(byn_dt_test[i])]))        
    else:
        krx_idx_test.append((etc_krx['종가'][list(etc_krx['날짜']).index(byn_dt_test[i])-hist_d_test[i]+1]-etc_krx['종가'][list(etc_krx['날짜']).index(byn_dt_test[i])]))

rate: 100%|████████████████████████████████████████████████████████████████████| 70596/70596 [00:32<00:00, 2190.38it/s]


In [13]:
## krx 데이터

train_data['krx_idx'] = krx_idx_train
test_data['krx_idx'] = krx_idx_test

krx_per_day = (etc_krx['종가'][0] - etc_krx['종가'][1226])/len(etc_krx)

test_krx_idx = copy.deepcopy(test_data['krx_idx'])
train_krx_idx = copy.deepcopy(train_data['krx_idx'])

test_krx_idx = (test_data['krx_idx'].max() + test_data['krx_idx'].min())/2 - test_krx_idx 
train_krx_idx = (train_data['krx_idx'].max() + train_data['krx_idx'].min())/2 - train_krx_idx 

test_krx_idx = test_krx_idx / krx_per_day
train_krx_idx = train_krx_idx / krx_per_day

for i in tqdm(range(len(test_krx_idx)), desc = 'rate', mininterval = 0.01):
    if test_krx_idx[i] < 0:
        test_krx_idx[i] = 0
        
for i in tqdm(range(len(train_krx_idx)), desc = 'rate', mininterval = 0.01):
    if train_krx_idx[i] < 0:
        train_krx_idx[i] = 0
        
test_krx_idx = test_krx_idx * (146/test_krx_idx.max())
train_krx_idx = train_krx_idx * (146/train_krx_idx.max())

test_krx_idx = np.trunc(test_krx_idx)
train_krx_idx = np.trunc(train_krx_idx)

train_data['krx_idx'] = train_krx_idx 
test_data['krx_idx'] = test_krx_idx 

rate: 100%|█████████████████████████████████████████████████████████████████| 681472/681472 [00:10<00:00, 65405.41it/s]


In [14]:
hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])]
train_data = pd.merge(train_data, kos_data, how = "left", on=['byn_dt'])
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data = pd.merge(test_data, kos_data, how = "left", on=['byn_dt'])
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)

L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

train_data = train_data.drop(['kospi', 'kospi_idx', 'kosdaq', 'kosdaq_idx', 'other', 'other_idx'], axis =1)
test_data = test_data.drop(['kospi', 'kospi_idx', 'kosdaq', 'kosdaq_idx', 'other', 'other_idx'], axis =1)

train_data.drop(["bnc_qty","tot_aet_amt","stk_par_pr","stk_p"], axis = 1, inplace = True)
test_data.drop(["bnc_qty","tot_aet_amt","stk_par_pr","stk_p"], axis = 1, inplace = True)

- test_data의 category를 가지는 feature에 대해 feature 별로 category에 따른 hist_d의 평균값을 list를 생성해 저장한다.

In [15]:
lsg_sgm_cd_test = []
for i in range(2,10):
    lsg_sgm_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'lsg_sgm_cd'] == i,'hist_d'].mean()))

tot_ivs_te_sgm_cd_test = []
for i in range(1,7):
    tot_ivs_te_sgm_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'tot_ivs_te_sgm_cd'] == i,'hist_d'].mean()))
    
n = [1,2,3,4,5,9,99]
tco_cus_grd_cd_test = []
for i in (n):
    tco_cus_grd_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'tco_cus_grd_cd'] == i,'hist_d'].mean()))      

mrz_btp_dit_cd_test = []
for i in range(1,17):
    mrz_btp_dit_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'mrz_btp_dit_cd'] == i,'hist_d'].mean()))      
    
n = [1,2,3,5,6,8,9,10,11,12,15,99]
mrz_pdt_tp_sgm_cd_test = []
for i in (n):
    mrz_pdt_tp_sgm_cd_test.append(test_data.loc[test_data.loc[:,'mrz_pdt_tp_sgm_cd'] == i,'hist_d'].mean()) 
    
    
n = [1,2,3,4,5,9,0,99]
ivs_icn_cd_test = []
for i in (n):
    ivs_icn_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'ivs_icn_cd'] == i,'hist_d'].mean()))


cus_age_stn_cd_test = []
for i in range(1,10):
    cus_age_stn_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'cus_age_stn_cd'] == i,'hist_d'].mean()))
    
btp_cfc_cd_test = []
for i in range(1,15):
    btp_cfc_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'btp_cfc_cd'] == i,'hist_d'].mean()))
    
n = [1,2,3,99]
mkt_pr_tal_scl_tp_cd_test = []
for i in (n):
    mkt_pr_tal_scl_tp_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'mkt_pr_tal_scl_tp_cd'] == i,'hist_d'].mean()))
    
n = [1,2,99]
stk_dit_cd_test = []
for i in (n):
    stk_dit_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'stk_dit_cd'] == i,'hist_d'].mean()))
    
    

n = [1,2,3,4,5,6]
cus_aet_stn_cd_test = []
for i in (n):
    cus_aet_stn_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'cus_aet_stn_cd'] == i,'hist_d'].mean()))
    
n = [1,2]
sex_dit_cd_test = []
for i in (n):
    sex_dit_cd_test.append(np.trunc(test_data.loc[test_data.loc[:,'sex_dit_cd'] == i,'hist_d'].mean()))

- train_data의 feature 별로 category에 따른 hold_d의 평균값을 list를 생성해 저장한다.
    - train_data의 경우 hist_d의 값이 명확치 않아 hold_d를 사용하였다.

In [16]:
lsg_sgm_cd_train = []
for i in range(2,10):
    lsg_sgm_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'lsg_sgm_cd'] == i,'hold_d'].mean()))
    
tot_ivs_te_sgm_cd_train = []
for i in range(1,7):
    tot_ivs_te_sgm_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'tot_ivs_te_sgm_cd'] == i,'hold_d'].mean()))    
    
n = [1,2,3,4,5,9,99]
tco_cus_grd_cd_train = []
for i in (n):
    tco_cus_grd_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'tco_cus_grd_cd'] == i,'hold_d'].mean()))      
    
mrz_btp_dit_cd_train = []
for i in range(1,17):
    mrz_btp_dit_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'mrz_btp_dit_cd'] == i,'hold_d'].mean()))      
    
n = [1,2,3,5,6,8,9,10,11,12,15,99]
mrz_pdt_tp_sgm_cd_train = []
for i in (n):
    mrz_pdt_tp_sgm_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'mrz_pdt_tp_sgm_cd'] == i,'hold_d'].mean())) 
    
n = [1,2,3,4,5,9,0,99]
ivs_icn_cd_train = []
for i in (n):
    ivs_icn_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'ivs_icn_cd'] == i,'hold_d'].mean()))

cus_age_stn_cd_train = []
for i in range(1,10):
    cus_age_stn_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'cus_age_stn_cd'] == i,'hold_d'].mean()))
    
btp_cfc_cd_train = []
for i in range(1,15):
    btp_cfc_cd_train.append(train_data.loc[train_data.loc[:,'btp_cfc_cd'] == i,'hold_d'].mean())
    
n = [1,2,3,99]
mkt_pr_tal_scl_tp_cd_train = []
for i in (n):
    mkt_pr_tal_scl_tp_cd_train.append(train_data.loc[train_data.loc[:,'mkt_pr_tal_scl_tp_cd'] == i,'hold_d'].mean())
    
n = [1,2,99]
stk_dit_cd_train = []
for i in (n):
    stk_dit_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'stk_dit_cd'] == i,'hold_d'].mean()))    
    
n = [1,2,3,4,5,6]
cus_aet_stn_cd_train = []
for i in (n):
    cus_aet_stn_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'cus_aet_stn_cd'] == i,'hold_d'].mean()))
    
n = [1,2]
sex_dit_cd_train = []
for i in (n):
    sex_dit_cd_train.append(np.trunc(train_data.loc[train_data.loc[:,'sex_dit_cd'] == i,'hold_d'].mean()))


- iem_krl_nm feature에 대해서도 위의 과정을 진행해준다.
    - iem_krl_nm의 경우 category의 종류가 3000개 가량 존재하는 특징이 있다.
- test_data와 train_data에 'sum'이라는 feature을 생성해준다.
- 위에서 생성한 feature의 category에 따른 평균값을 저장한 list를 데이터의 featrue의 category에 따라 'sum' feature에 더해준다.
- test_data와 train_data에 'diff'라는 feature을 생성해준다.
- iem_krl_nm feature의 category에 따른 평균값을 저장한 list를 데이터의 featrue의 category에 따라 'diff' feature에 더해준다.

In [17]:
iem_krl_nm_test = []

for i in range(0,3075):
    iem_krl_nm_test.append(np.trunc(test_data.loc[test_data.loc[:,'iem_krl_nm'] == i,'hist_d'].mean()))

iem_krl_nm_train = []

for i in range(0,3075):
    iem_krl_nm_train.append(np.trunc(train.loc[train_data.loc[:,'iem_krl_nm'] == i,'hold_d'].mean()))


test_data.loc[:,'sum'] = 0
train_data.loc[:,'sum'] = 0
n = [2,3,4,5,6,7,8,9]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'lsg_sgm_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'lsg_sgm_cd'] == v,"sum"]+ lsg_sgm_cd_test[i]

n = [1,2,3,4,5,6]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'tot_ivs_te_sgm_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'tot_ivs_te_sgm_cd'] == v,"sum"]+ tot_ivs_te_sgm_cd_test[i]

n = [1,2,3,4,5,9,99]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'tco_cus_grd_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'tco_cus_grd_cd'] == v,"sum"]+ tco_cus_grd_cd_test[i]

n = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'mrz_btp_dit_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'mrz_btp_dit_cd'] == v,"sum"]+ mrz_btp_dit_cd_test[i]

n = [1,2,3,5,6,8,9,10,11,12,15,99]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'mrz_pdt_tp_sgm_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'mrz_pdt_tp_sgm_cd'] == v,"sum"]+ mrz_pdt_tp_sgm_cd_test[i]

n = [1,2,3,4,5,9,0,99]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'ivs_icn_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'ivs_icn_cd'] == v,"sum"]+ ivs_icn_cd_test[i]

n = [1,2,3,4,5,6,7,8,9]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'cus_age_stn_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'cus_age_stn_cd'] == v,"sum"]+ cus_age_stn_cd_test[i]

n = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'btp_cfc_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'btp_cfc_cd'] == v,"sum"]+ btp_cfc_cd_test[i]

n = [1,2,3,99]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'mkt_pr_tal_scl_tp_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'mkt_pr_tal_scl_tp_cd'] == v,"sum"]+ mkt_pr_tal_scl_tp_cd_test[i]

n = [1,2,99]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'stk_dit_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'stk_dit_cd'] == v,"sum"]+ stk_dit_cd_test[i]

n = [1,2,3,4,5,6]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'cus_aet_stn_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'cus_aet_stn_cd'] == v,"sum"]+ cus_aet_stn_cd_test[i]
n = [1,2]
for i,v in enumerate(n):
    test_data.loc[test_data.loc[:,'sex_dit_cd'] == v,"sum"] = test_data.loc[test_data.loc[:,'sex_dit_cd'] == v,"sum"]+ sex_dit_cd_test[i]
n = [2,3,4,5,6,7,8,9]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'lsg_sgm_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'lsg_sgm_cd'] == v,"sum"]+ lsg_sgm_cd_train[i]

n = [1,2,3,4,5,6]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'tot_ivs_te_sgm_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'tot_ivs_te_sgm_cd'] == v,"sum"]+ tot_ivs_te_sgm_cd_train[i]

n = [1,2,3,4,5,9,99]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'tco_cus_grd_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'tco_cus_grd_cd'] == v,"sum"]+ tco_cus_grd_cd_train[i]

n = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'mrz_btp_dit_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'mrz_btp_dit_cd'] == v,"sum"]+ mrz_btp_dit_cd_train[i]

n = [1,2,3,5,6,8,9,10,11,12,15,99]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'mrz_pdt_tp_sgm_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'mrz_pdt_tp_sgm_cd'] == v,"sum"]+ mrz_pdt_tp_sgm_cd_train[i]

n = [1,2,3,4,5,9,0,99]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'ivs_icn_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'ivs_icn_cd'] == v,"sum"]+ ivs_icn_cd_train[i]

n = [1,2,3,4,5,6,7,8,9]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'cus_age_stn_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'cus_age_stn_cd'] == v,"sum"]+ cus_age_stn_cd_train[i]

n = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'btp_cfc_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'btp_cfc_cd'] == v,"sum"]+ btp_cfc_cd_train[i]

n = [1,2,3,99]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'mkt_pr_tal_scl_tp_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'mkt_pr_tal_scl_tp_cd'] == v,"sum"]+ mkt_pr_tal_scl_tp_cd_train[i]

n = [1,2,99]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'stk_dit_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'stk_dit_cd'] == v,"sum"]+ stk_dit_cd_train[i]

n = [1,2,3,4,5,6]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'cus_aet_stn_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'cus_aet_stn_cd'] == v,"sum"]+ cus_aet_stn_cd_train[i]
n = [1,2]
for i,v in enumerate(n):
    train_data.loc[train_data.loc[:,'sex_dit_cd'] == v,"sum"] = train_data.loc[train_data.loc[:,'sex_dit_cd'] == v,"sum"]+ sex_dit_cd_train[i]
for i in range(0,3075):
    if np.isnan(iem_krl_nm_test[i]):
        continue
    test_data.loc[test_data.loc[:,'iem_krl_nm'] == i,'sum']= test_data.loc[test_data.loc[:,'iem_krl_nm'] == i,'sum']+ np.trunc(iem_krl_nm_test[i])
    
for i in range(0,3075):
    if np.isnan(iem_krl_nm_train[i]):
        continue
    train_data.loc[train_data.loc[:,'iem_krl_nm'] == i,'sum']=train_data.loc[train_data.loc[:,'iem_krl_nm'] == i,'sum']+ np.trunc(iem_krl_nm_train[i])

    
    
    
for i in range(0,3075):
    if np.isnan(iem_krl_nm_test[i]):
        continue
    test_data.loc[test_data.loc[:,'iem_krl_nm'] == i,'diff']= np.trunc(iem_krl_nm_test[i])
    
for i in range(0,3075):
    if np.isnan(iem_krl_nm_train[i]):
        continue
    train_data.loc[train_data.loc[:,'iem_krl_nm'] == i,'diff']=np.trunc(iem_krl_nm_train[i])

1. train_data에 대해서 hold_d의 값을 30 ~ 60, 60 ~ 100, 100 ~ 147,그리고 147초과하는 범위로 구간을 나눈다.
2. 나눈 구간 별로 앞서 생성한 sum feature의 max값은 550, 465, 410, 879이다
3. 147미만의 3구역에서는 sum feature의 max값을 hold_d * sum에 나눠준다.
    - (hold_d * sum)/(sum feature의 max)가 0보다는 크며 hold_d보다 작은 값을 가진다.
    - 3번의 계산 결과를 hold_d - hist_d라고 가정한다.
4. 3번의 계산 결과인 (hold_d - hist_d)를 hold_d에 빼준 결과를 hist_d에 넣어준다.
    - sum feature값이 클 수록 hold_d - hist_d가 클거라는 가정을 통해서 나온 결과이다.
5. 147이상의 구역에서는 max값인 879보다 작은 720을 나누어 주었다.
    - 879값을 그대로 사용하기에는 min값과 max값의 차이가 다른 구간에 비해 커져서 적정선의 값인 720으로 나누어주었다.
6. 다음은 (hold_d - hist_d) >146인 경우에는 hold_d - 146으로 일괄 처리한다.
7. train_label를 생성한다.

In [18]:
for i in range(30,60):
    train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'hist_d'] = train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'hold_d'] -np.trunc(train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'sum']*i/550)
for i in range(60,100):
    train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'hist_d'] = train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'hold_d'] -np.trunc(train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'sum']*i/465)
for i in range(100,147):
    train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'hist_d'] = train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'hold_d'] -np.trunc(train_data.loc[train_data[train_data.loc[:,'hold_d']== i].index,'sum']*i/410)

train_data.loc[train_data[train_data.loc[:,'hold_d']> 146].index,'hist_d'] = train_data.loc[train_data[train_data.loc[:,'hold_d']> 146].index,'hold_d'] -np.trunc(train_data.loc[train_data[train_data.loc[:,'hold_d']> 146].index,'sum']*146/720) 
train_data.loc[train_data.loc[train_data[train_data.loc[:,'hold_d']> 146].index,'hist_d'][(train_data.loc[train_data[train_data.loc[:,'hold_d']> 146].index,'sum']*146/720)>146].index,'hist_d'] =train_data.loc[train_data.loc[train_data[train_data.loc[:,'hold_d']> 146].index,'hist_d'][(train_data.loc[train_data[train_data.loc[:,'hold_d']> 146].index,'sum']*146/720)>146].index,'hold_d']- 146
train_data.loc[train_data.loc[train_data.loc[:,'hold_d']-train_data.loc[:,'hist_d']>146].index,'hist_d'] =train_data.loc[train_data.loc[train_data.loc[:,'hold_d']-train_data.loc[:,'hist_d']>146].index,'hold_d']-146
train_data.hist_d = np.trunc(train_data.hist_d)
train_data.hist_d = train_data.hist_d.fillna(0)
train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)


test_data = np.trunc(test_data)
train_data = np.trunc(train_data)

- 마지막으로 기존 kos_idx, krx_idx은 hold_d - hist_d 값을 나타내는 지표이므로 이에 hist_d 값을 더해줌

In [19]:
train_data['kos_idx'] = train_data['kos_idx'] + train_data['hist_d']
test_data['kos_idx'] = test_data['kos_idx'] + test_data['hist_d']

train_data['krx_idx'] = train_data['krx_idx'] + train_data['hist_d']
test_data['krx_idx'] = test_data['krx_idx'] + test_data['hist_d']


In [20]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.loc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.loc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 3500,
                          learning_rate= 0.01,
                          num_leaves = 31)

    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=150,
              verbose=500)
    
    models.append(model)

Training until validation scores don't improve for 150 rounds
[500]	valid_0's rmse: 4.97531	valid_0's l2: 24.7538
[1000]	valid_0's rmse: 4.8384	valid_0's l2: 23.4101
[1500]	valid_0's rmse: 4.8268	valid_0's l2: 23.298
Early stopping, best iteration is:
[1372]	valid_0's rmse: 4.82651	valid_0's l2: 23.2952
Training until validation scores don't improve for 150 rounds
[500]	valid_0's rmse: 2.29091	valid_0's l2: 5.24828
[1000]	valid_0's rmse: 2.19433	valid_0's l2: 4.81508
[1500]	valid_0's rmse: 2.19031	valid_0's l2: 4.79745
[2000]	valid_0's rmse: 2.18877	valid_0's l2: 4.79071
Early stopping, best iteration is:
[1875]	valid_0's rmse: 2.18825	valid_0's l2: 4.78845
Training until validation scores don't improve for 150 rounds
[500]	valid_0's rmse: 2.19804	valid_0's l2: 4.83138
[1000]	valid_0's rmse: 2.10926	valid_0's l2: 4.44899
Early stopping, best iteration is:
[1106]	valid_0's rmse: 2.10273	valid_0's l2: 4.42148
Training until validation scores don't improve for 150 rounds
[500]	valid_0's r

최종 결과 data 파일에 생성

In [21]:
result = []
for i in models:
    result.append(i.predict(test_data))
predict = np.mean(result, axis = 0)
submission["hold_d"] = np.round(predict)
submission.loc[submission["hold_d"]-test.hist_d >146,'hold_d'] = test.loc[submission["hold_d"]-test.hist_d >146,'hist_d']+138
submission.loc[(submission["hold_d"]-test.hist_d) <1,'hold_d'] = test.loc[submission["hold_d"]-test.hist_d<1,'hist_d']+9
submission.to_csv("data/dacon_1011_R2.csv", index = False)